In [5]:
import json
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
companies_data_frame = pd.read_csv("./csv/companies.csv")
companies_data_frame

,stock_code,name_vn,name_en,name_short,post_to,C
0,A32,Công ty cổ phần 32,32 Joint Stock Company,A32,UPCOM,3
1,AAA,Công ty cổ phần Nhựa An Phát Xanh,An Phat Bioplastics Joint Stock Company,AAA,HOSE,1
2,AAM,Công ty Cổ phần Thủy sản Mekong,Mekong Fisheries Joint Stock Company,AAM,HOSE,1
3,AAS,Công ty cổ phần Chứng khoán SmartInvest,Smart Invest Securities Joint Stock Company,AAS,UPCOM,3
4,AAT,Công ty cổ phần Tập đoàn Tiên Sơn Thanh Hóa,Tien Son Thanh Hoa Joint Stock Company,AAT,HOSE,1
...,...,...,...,...,...,...
2285,XPH,CTCP Xà phòng Hà Nội,CTCP Xa phong Ha Noi,XPH Joint Stock Company,UPCOM,3
2286,YBC,CTCP Xi măng và Khoáng sản Yên Bái,Công ty CP Xi măng và khoáng sản Yên Bái,YBC Joint Stock Company,UPCOM,3
2287,YBM,Công ty cổ phần Khoáng sản Công nghiệp Yên Bái,Yen Bai Industry Mineral Joint Stock Company,YBM Joint Stock Company,HOSE,1
2288,YEG,Công ty cổ phần Tập đoàn Yeah1,Shares of Yeah1 Group Joint Stock Company,YEG Joint Stock Company,HOSE,1


In [7]:
def analyse(symbol):
    history_data_frame = pd.read_csv(f"./csv/history/all/{symbol}.csv")
    # Past
    history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
    history_data_frame["price_three_days_ago_difference"] = round(history_data_frame["close"] - history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_three_days_ago_return"] = round(history_data_frame["price_three_days_ago_difference"] / history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_two_days_ago"] = history_data_frame["close"].shift(2)
    history_data_frame["price_yesterday"] = history_data_frame["close"].shift(1)
    # Future
    history_data_frame["price_tomorrow"] = history_data_frame["close"].shift(-1)
    history_data_frame["price_tomorrow_difference"] = round(history_data_frame["price_tomorrow"] - history_data_frame["close"], 2)
    history_data_frame["price_tomorrow_return"] = round(history_data_frame["price_tomorrow_difference"] / history_data_frame["close"], 2)
    # Direction
    history_data_frame["direction"] = [1 if history_data_frame.loc[ei, "price_tomorrow_difference"] > 0 else -1 for ei in history_data_frame.index]
    # Moving Average
    history_data_frame["MA5"] = round(history_data_frame["close"].rolling(5).mean(), 2)
    history_data_frame["MA10"] = round(history_data_frame["close"].rolling(10).mean(), 2)
    history_data_frame["MA20"] = round(history_data_frame["close"].rolling(20).mean(), 2)
    history_data_frame["MA50"] = round(history_data_frame["close"].rolling(50).mean(), 2)
    history_data_frame["MA100"] = round(history_data_frame["close"].rolling(100).mean(), 2)
    history_data_frame["MA200"] = round(history_data_frame["close"].rolling(200).mean(), 2)
    # Save to File
    plt.plot(history_data_frame[['close', 'MA50', 'MA200']].tail(200))
    plt.legend(['close', 'MA50', 'MA200'], loc='upper left')
    plt.savefig(f"./images/{symbol}.png")
    plt.clf()
    history_data_frame.to_csv(f"./csv/analyse/{symbol}.csv",index=False)

In [8]:
vn30_file = open("./vn30.json", 'r')
vn30 : list[str] = json.load(vn30_file)
for symbol in vn30:
    print(symbol)
    analyse(symbol)

ACB
BCM
BID
BVH
CTG
FPT
GAS
GVR
HDB
HPG
MBB
MSN
MWG
PLX
POW
SAB
SHB
SSB
SSI
STB
TCB
TPB
VCB
VHM
VIB
VIC
VJC
VNM
VPB
VRE


<Figure size 640x480 with 0 Axes>